In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

In [ ]:
E = 210E9  # Pa
FApplied = 10000
BarLenght = 0.5

xi = sp.symbols('xi')

NumElements = 1

Tractions = None

Elements = []
NodeNumbers = []

def Meshing(N,L):
    n0 = np.linspace(0,L,(2*N)+1)
    np.append(n0,L)
    for i in range(N):
        Elements.append([n0[i*2],n0[i*2+1],n0[i*2+2]])
        NodeNumbers.append([i*2,i*2+1,i*2+2])
    pass

def ReturnXForElement(ElementNum):
    N1 = -1/2*xi*(1-xi)
    N2 = 1/2*xi*(1+xi)
    N3 = (1-xi)*(1+xi)
    x1 = Elements[ElementNum][0]
    x2 = Elements[ElementNum][2]
    x3 = Elements[ElementNum][1]
    return N1*x1 + N2*x2 + N3*x3
   

def AreaForElement(ElementNum):
    return sp.simplify(0.0001 + 0.0001*ReturnXForElement(ElementNum))

def BuildTractionVector(FApplied,NodeNum,Tractions):
    
    size = NodeNumbers[len(NodeNumbers)-1][2]
    if Tractions==None:
        Tractions = np.zeros([size+1])
    arr = np.array(NodeNumbers)
    if NumElements==1:
        indx = np.where(arr == NodeNum)[0]
    else:
        indx = np.where(arr == NodeNum)[0][0]
        
    print(indx)
    Tractions = sp.Matrix(Tractions)
    Tractions[NodeNum] = Tractions[NodeNum] + FApplied
    return  sp.Matrix(Tractions)
 
def ReturnNMatrix():
    N1 = -1/2*xi*(1-xi)
    N2 = 1/2*xi*(1+xi)
    N3 = (1-xi)*(1+xi)
    return sp.Matrix([N1,N2,N3] )
    
def BuildSmallStiffi(ElementNum):
    N1 = -1/2*xi*(1-xi)
    N2 = 1/2*xi*(1+xi)
    N3 = (1-xi)*(1+xi)
    N = [N1,N2,N3]  ### may need to cahnge order
    x = ReturnXForElement(ElementNum)
    dxdxi = sp.derive_by_array(x,xi)
    dxidx = 1/dxdxi
    #print('dxidx',dxidx)
    B = dxidx*sp.derive_by_array(N,xi);
    #print('B:',B)
    Bmat = sp.Matrix(B).transpose()
    BTEB = Bmat.transpose()*E*Bmat
    
    integrandStiffnessMatrix = BTEB*AreaForElement(ElementNum)*dxdxi
    #gaus quad
    StiffnessMatrix = 1.0*integrandStiffnessMatrix.subs({xi:-1/3**0.5}) + 1.0*integrandStiffnessMatrix.subs({xi:1/3**0.5})
    
    return sp.simplify(StiffnessMatrix)
    
def AssembleStiffnessMatrix():
    size = NodeNumbers[len(NodeNumbers)-1][2]+1
    StiffnessMatrix = np.zeros([size,size])
    
    for idx,i in enumerate(Elements):
        Stiffnessi1 = np.zeros([size,size])
        M = BuildSmallStiffi(idx)
        
        Stiffnessi1[NodeNumbers[idx][0]][NodeNumbers[idx][0]] = M[4]
        Stiffnessi1[NodeNumbers[idx][0]][NodeNumbers[idx][1]] = M[1]
        Stiffnessi1[NodeNumbers[idx][0]][NodeNumbers[idx][2]] = M[5]
        Stiffnessi1[NodeNumbers[idx][1]][NodeNumbers[idx][0]] = M[3]
        Stiffnessi1[NodeNumbers[idx][1]][NodeNumbers[idx][1]] = M[0]
        Stiffnessi1[NodeNumbers[idx][1]][NodeNumbers[idx][2]] = M[2]
        Stiffnessi1[NodeNumbers[idx][2]][NodeNumbers[idx][0]] = M[7]
        Stiffnessi1[NodeNumbers[idx][2]][NodeNumbers[idx][1]] = M[6]
        Stiffnessi1[NodeNumbers[idx][2]][NodeNumbers[idx][2]] = M[8]
        StiffnessMatrix = StiffnessMatrix + Stiffnessi1
    
    return sp.Matrix(StiffnessMatrix)
  

#initializing the meshing    
Meshing(NumElements, BarLenght)
#Building of traction vector global
Tractions = BuildTractionVector(FApplied,NumElements*2-1,Tractions)


    
StiffnessMatrix = AssembleStiffnessMatrix()
print('Elements positions:')
print(Elements)
print('Node numbers:')
print(NodeNumbers)
print('Traction vector')
print(Tractions)
print('Big Stiffness Matrix:')
display(StiffnessMatrix)

StiffnessMatrixReduced = StiffnessMatrix[1:,1:]


solution = StiffnessMatrixReduced.solve(Tractions[1:,0]-StiffnessMatrix[1:,0]*0)


q =[0]
for i in solution:
    q.append(i)

print('nodal displacements:')
print(q)